In [1]:
import os, json, time, llm_utils, config, re
import pandas as pd
from tqdm import tqdm
from azure.storage.blob import BlobServiceClient
from Arxiv_API import Arxiv_API

keys=config.get_all_variables()
rootdir=keys['rootdir']
gemini_key=keys['gemini_key']
connection_str=keys['connection_str']

/Users/tengli/Python/chatGPT/LLMVenv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data Collection

In [ ]:
# Download the PDF from Azure storage account
blob_service_client = BlobServiceClient.from_connection_string(conn_str=connection_str)

with open("../SampleData/Data.pdf", "wb") as pdf_file:
        blob_data = blob_service_client.get_blob_client(container="samplepicture", blob=f"Data/Article_2306.00251v1.pdf").download_blob()
        pdf_file.write(blob_data.readall())

In [89]:
# Or download pdf from Arxiv and save as metadata
data=Arxiv_API(search=['math','physics','cs','econ','eess','q-bio','q-fin','stat'], start_date='2000-01-01', end_date='2024-12-31', start=0, sampling_unit_size=50, max_results=100, downloadstuff=True).df

2025-01-22 15:56:53,689 - INFO - Feed title of metadata 0: ArXiv Query: search_query=cat:math*&amp;id_list=&amp;start=0&amp;max_results=100
2025-01-22 15:56:53,690 - INFO - Feed last update of metadata 0: 2025-01-22T00:00:00-05:00
2025-01-22 15:56:53,690 - INFO - totalResults for query 0: 677553
2025-01-22 15:56:53,690 - INFO - totalResults for query 0: 100
2025-01-22 15:56:53,690 - INFO - totalResults for query 0: 0
2025-01-22 15:56:53,690 - INFO - Generating e-print metadata for Article http://arxiv.org/abs/2012.13850v1
2025-01-22 15:56:53,691 - INFO - PDF from metadata list 0 article 0 already exists.
2025-01-22 15:56:54,374 - INFO - Generating e-print metadata for Article http://arxiv.org/abs/2412.05250v1
2025-01-22 15:56:54,374 - INFO - PDF from metadata list 0 article 1 already exists.
2025-01-22 15:56:55,954 - INFO - Generating e-print metadata for Article http://arxiv.org/abs/2201.06408v1
2025-01-22 15:56:55,954 - INFO - PDF from metadata list 0 article 2 already exists.
2025-0

In [2]:
# Or load the data from metadata.json
with open('../SampleData/metadata.json') as outfile:
    outputs=json.load(outfile)

metadata=pd.DataFrame(outputs)
metadata['Authors']=metadata['Authors'].apply(lambda x: [author.strip() for author in x.split(',')])
metadata['Primary_Cat']=metadata['Primary_Cat'].apply(lambda x: x.split('.')[0])

display(metadata)

,ID,PublishDate,Title,Authors,Journal_Ref,Comment,Abstract,Content,Primary_Cat,Category,PDF_link,file_path
0,2012.13850v1,2020-12-27T02:25:29Z,Generalized spaces for constructive algebra,[Ingo Blechschmidt],No Journal_Ref,"Chapter for an upcoming collection ""Proof and ...",The purpose of this contribution is to give a ...,arXiv:2012.13850v1 [math.LO] 27 Dec 2020Dece...,math,No Category,http://arxiv.org/pdf/2012.13850v1,/Users/tengli/Python/chatGPT/SampleData/math/A...
1,2412.05250v1,2024-12-06T18:30:30Z,Constructing projective modules,[Aravind Asok],No Journal_Ref,138 pages; comments welcome!,We discuss elements of a social history of the...,arXiv:2412.05250v1 [math.HO] 6 Dec 2024Const...,math,No Category,http://arxiv.org/pdf/2412.05250v1,/Users/tengli/Python/chatGPT/SampleData/math/A...
2,2201.06408v1,2022-01-17T13:48:30Z,Quantalic spectra of semirings,[Graham Manuell],No Journal_Ref,My PhD thesis from 2019/2020. 117 pages,Spectrum constructions appear throughout mathe...,Quantalic spectra of\nsemirings\nGraham Manuel...,math,No Category,http://arxiv.org/pdf/2201.06408v1,/Users/tengli/Python/chatGPT/SampleData/math/A...
3,0409250v3,2004-09-15T14:01:51Z,Von Neumann coordinatization is not first-order,[Friedrich Wehrung],"Journal of Mathematical Logic 6, no. 1 (2006) ...",No Comment,"A lattice L is coordinatizable, if it is isomo...",arXiv:math/0409250v3 [math.GM] 28 Jan 2006VO...,math,No Category,http://arxiv.org/pdf/math/0409250v3,/Users/tengli/Python/chatGPT/SampleData/math/A...
4,0509245v2,2005-09-11T20:22:45Z,Analysis in J_2,[Nik Weaver],No Journal_Ref,31 pages,This is an expository paper in which I explain...,arXiv:math/0509245v2 [math.LO] 12 Sep 2005AN...,math,No Category,http://arxiv.org/pdf/math/0509245v2,/Users/tengli/Python/chatGPT/SampleData/math/A...
...,...,...,...,...,...,...,...,...,...,...,...,...
395,2012.05405v1,2020-12-10T01:53:20Z,PoolTestR: An R package for estimating prevale...,"[Angus McLure, Ben O'Neill, Helen Mayfield, Co...","Environ. Model. Softw. Volume 145, November 20...","18 pages, 2 figures, 1 table, 2 boxes","Pooled testing (also known as group testing), ...",PoolTest R: An R package for estimating \nprev...,stat,No Category,http://arxiv.org/pdf/2012.05405v1,/Users/tengli/Python/chatGPT/SampleData/stat/A...
396,2112.02950v1,2021-12-06T11:47:22Z,Bayesian Estimation Approach for Linear Regres...,"[Solmaz Seifollahi, Kaniav Kamary, Hossein Bev...",No Journal_Ref,No Comment,Univariate and multivariate general linear reg...,Bayesian Estimation Approach for Linear Regres...,stat,No Category,http://arxiv.org/pdf/2112.02950v1,/Users/tengli/Python/chatGPT/SampleData/stat/A...
397,2102.03106v1,2021-02-05T11:12:18Z,ROBustness In Network (robin): an R package fo...,"[Valeria Policastro, Dario Righelli, Annamaria...",No Journal_Ref,No Comment,"In network analysis, many community detection ...",1 \nROBustness In Network (robin): an R \np...,stat,No Category,http://arxiv.org/pdf/2102.03106v1,/Users/tengli/Python/chatGPT/SampleData/stat/A...
398,2102.12290v3,2021-02-24T13:55:30Z,Smooth Online Parameter Estimation for time va...,"[Anass El Yaagoubi Bourakna, Marco Pinto, Norb...",No Journal_Ref,No Comment,Multivariate time series data appear often as ...,Statistics and Its Interface Volume 0 (2019) 1...,stat,No Category,http://arxiv.org/pdf/2102.12290v3,/Users/tengli/Python/chatGPT/SampleData/stat/A...


# Define functions to run models

In [3]:

def clean_text(text):
    # Remove invalid or non-printable characters, but keep Unicode printable ones
    return re.sub(r'[^\x20-\x7E\xA0-\uFFFF]', '', text)

def getEachRes(user_prompt,row):
        input_max_tokens = 1000
        raw_text = user_prompt.format(row['Content'][0:input_max_tokens])
        return raw_text



def GetTitleAuthor_gpt():
    engine="gpt-3.5-turbo"
    system_prompt = "Please extract the title and author names:"
    system_prompt_tokens = llm_utils.count_tokens(text=system_prompt, engine=engine)

    def getEachRes(row):
        input_max_tokens = 100

        raw_text = row['Content']
        user_prompt = llm_utils.get_tokens_between_indices(
            text=raw_text, 
            engine=engine, 
            max_tokens=input_max_tokens)
        print(user_prompt)
        
        response=llm_utils.get_gpt_response(user_prompt=user_prompt, 
                                            system_prompt=system_prompt,
                                            engine=engine,
                                            max_completion=500)
        print("ChatGPT Response:", response)
        return response

    results = []
    for i in tqdm(range(len(metadata)), desc="Processing rows"):
        row = metadata.iloc[i]
        result = getEachRes(row)
        results.append(result)

    # After all rows are processed, write the accumulated results
    with open("all_results.json", 'w') as f:
        json.dump(results, f)



def GetTitleAuthor_gemini(metadata, user_prompt, api_key):
    results = []
    for i in tqdm(range(len(metadata)), desc="Processing rows"):
        row = metadata.iloc[i]
        raw_text = getEachRes(user_prompt, row)
        result=llm_utils.get_gemini_response(user_prompt=raw_text, api_key=api_key)
        #print(result)
        result=json.loads(clean_text(result))[0] #this is the dictionary in the json response
        true_res=metadata.iloc[i,][['ID','Primary_Cat','Title','Authors']].rename({'Title': 'TrueTitle', 'Authors': 'TrueAuthors'}).to_dict()
        result.update(true_res)
        results.append(result)
        time.sleep(10) 
    
    # After all rows are processed, write the accumulated results
    with open(os.path.join(rootdir, "results", "gemini_results.json"), 'w') as f:
        json.dump(results, f)



def GetTitleAuthor_llama(metadata, user_prompt):
    results = []
    for i in tqdm(range(len(metadata)), desc="Processing rows"):
        row = metadata.iloc[i]
        raw_text = getEachRes(user_prompt, row)
        result=llm_utils.get_llama_response(user_prompt=raw_text)
        result=json.loads(clean_text(result))
        true_res=metadata.iloc[i,][['ID','Primary_Cat','Title','Authors']].rename({'Title': 'TrueTitle', 'Authors': 'TrueAuthors'}).to_dict()
        result.update(true_res)
        #print(result)
        results.append(result)

    # After all rows are processed, write the accumulated results
    with open(os.path.join(rootdir, "results", "llama_results.json"), 'w') as f:
        json.dump(results, f)



# create prompt message
prompt_message = "Extract the title and author names from this research paper: \"{}\". If a field cannot be found, return 'NA'. "


In [4]:
GetTitleAuthor_gemini(metadata=metadata, user_prompt=prompt_message, api_key=gemini_key)

with open(os.path.join(rootdir, 'results', 'gemini_results.json')) as outfile:
    outputs=json.load(outfile)
df_gemini = pd.json_normalize(outputs)
display(df_gemini)

Processing rows: 100%|██████████| 400/400 [1:12:41<00:00, 10.90s/it]


,Authors,Title,ID,Primary_Cat,TrueTitle,TrueAuthors
0,[Ingo Blechschmidt],Generalized spaces for constructive algebra,2012.13850v1,math,Generalized spaces for constructive algebra,[Ingo Blechschmidt]
1,[Aravind Asok],Constructing projective modules,2412.05250v1,math,Constructing projective modules,[Aravind Asok]
2,[Graham Manuell],Quantalic spectra of semirings,2201.06408v1,math,Quantalic spectra of semirings,[Graham Manuell]
3,[FRIEDRICH WEHRUNG],VON NEUMANN COORDINATIZATION IS NOT FIRST-ORDER,0409250v3,math,Von Neumann coordinatization is not first-order,[Friedrich Wehrung]
4,[Nik Weaver],Analysis in J2,0509245v2,math,Analysis in J_2,[Nik Weaver]
...,...,...,...,...,...,...
395,"[Angus McLure, Ben O’Neill, Helen Mayfield, Co...",PoolTest R: An R package for estimating preval...,2012.05405v1,stat,PoolTestR: An R package for estimating prevale...,"[Angus McLure, Ben O'Neill, Helen Mayfield, Co..."
396,"[SOLMAZ SEIFOLLAHI, KANIAV KAMARY, HOSSEIN BEV...",Bayesian Estimation Approach for Linear Regres...,2112.02950v1,stat,Bayesian Estimation Approach for Linear Regres...,"[Solmaz Seifollahi, Kaniav Kamary, Hossein Bev..."
397,"[Valeria Policastro, Dario Righelli, Annamaria...",ROBustness In Network (robin): an R package fo...,2102.03106v1,stat,ROBustness In Network (robin): an R package fo...,"[Valeria Policastro, Dario Righelli, Annamaria..."
398,"[Anass El Yaagoubi Bourakna, Marco Pinto, Norb...",Smooth Online Parameter Estimation for time va...,2102.12290v3,stat,Smooth Online Parameter Estimation for time va...,"[Anass El Yaagoubi Bourakna, Marco Pinto, Norb..."


In [10]:
GetTitleAuthor_llama(metadata=metadata, user_prompt=prompt_message)

with open(os.path.join(rootdir, 'results', 'llama_results.json')) as outfile:
    outputs=json.load(outfile)
df_llama = pd.json_normalize(outputs)
display(df_llama)

Processing rows: 100%|██████████| 400/400 [24:46<00:00,  3.72s/it]


,Title,Authors,ID,Primary_Cat,TrueTitle,TrueAuthors
0,Generalized spaces for constructive algebra,[Ingo Blechschmidt],2012.13850v1,math,Generalized spaces for constructive algebra,[Ingo Blechschmidt]
1,Constructing projective modules,[Aravind Asok],2412.05250v1,math,Constructing projective modules,[Aravind Asok]
2,Quantalic spectra of semirings,[Graham Manuell],2201.06408v1,math,Quantalic spectra of semirings,[Graham Manuell]
3,Von Neumann Coordinatization Is Not First-Order,[Friedrich Wehrung],0409250v3,math,Von Neumann coordinatization is not first-order,[Friedrich Wehrung]
4,Analysis in J2,[Nik Weaver],0509245v2,math,Analysis in J_2,[Nik Weaver]
...,...,...,...,...,...,...
395,PoolTest R: An R package for estimating preval...,"[Angus McLure, Ben O’Neill, Helen Mayfield, Co...",2012.05405v1,stat,PoolTestR: An R package for estimating prevale...,"[Angus McLure, Ben O'Neill, Helen Mayfield, Co..."
396,Bayesian Estimation Approach for Linear Regres...,"[SOLMAZ SEIFOLLAHI, K ANIAV KAMARY, H OSSEIN B...",2112.02950v1,stat,Bayesian Estimation Approach for Linear Regres...,"[Solmaz Seifollahi, Kaniav Kamary, Hossein Bev..."
397,ROBustness In Network (robin): an R package fo...,"[Valeria Policastro, Dario Righelli, Annamaria...",2102.03106v1,stat,ROBustness In Network (robin): an R package fo...,"[Valeria Policastro, Dario Righelli, Annamaria..."
398,Smooth Online Parameter Estimation for time va...,"[Anass El Yaagoubi Bourakna, Marco Pinto, Norb...",2102.12290v3,stat,Smooth Online Parameter Estimation for time va...,"[Anass El Yaagoubi Bourakna, Marco Pinto, Norb..."
